In [ ]:
from zipfile import ZipFile
ref = ZipFile('/content/animal-image-dataset-90-different-animals.zip', 'r')
ref.extractall('/content')
ref.close()

In [ ]:
import keras
import tensorflow as tf
import os, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from glob import glob
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras.layers import Dropout, BatchNormalization, Flatten, Dense
from tensorflow.keras.models import Sequential


In [ ]:
np.__version__

In [3]:
def get_image_paths(root_dir,num_images=None):
  all_images = []
  for extension in ["*.jpg", "*.jpeg", "*.png"]:
    all_images.extend(glob(os.path.join(root_dir,'**',extension),recursive=True))
  if num_images is None:
    return all_images
  else:
    return random.sample(all_images,min(num_images,len(all_images)))
img_list = get_image_paths("/kaggle/input/animal-image-dataset-90-different-animals/animals/animals")

In [4]:
def display_images(img_list):
    plt.figure(figsize=(15, 6))
    for i, img_path in enumerate(img_list):
        img = image.load_img(img_path)
        img = image.img_to_array(img, dtype=np.uint8)
        plt.subplot(2, 5, i + 1)
        # plt.imshow(img.squeeze())
        plt.imshow(img)
        plt.axis("off")
        plt.title(f"Image {i+1}")
    plt.tight_layout()
    plt.show()

In [5]:
display_images(np.random.choice(img_list,10))

In [6]:
dir_path = "/kaggle/input/animal-image-dataset-90-different-animals/animals/animals"

In [7]:
train_gen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip = True,
    rotation_range=30,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    validation_split = 0.3
)
val_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split = 0.3
)

In [35]:
train_generator = train_gen.flow_from_directory(
    dir_path,
    target_size=(224,224),
    batch_size=32,
    subset='training',
    class_mode = 'sparse'
)
val_ds = val_gen.flow_from_directory(
    dir_path,
    target_size=(224,224),
    batch_size=32,
    subset='validation',
    class_mode = 'sparse'
)

Found 3780 images belonging to 90 classes.
Found 1620 images belonging to 90 classes.


In [36]:
from tensorflow.keras.applications import ResNet50V2

In [48]:
from tensorflow.keras.applications import MobileNet

In [56]:
base_model = MobileNet(
    weights='imagenet',
    input_shape=(224,224,3),
    include_top = False
)

In [37]:
from tensorflow.keras.applications import InceptionV3

base_model = InceptionV3(
    weights='imagenet',
    input_shape=(224,224,3),
    include_top = False
)

In [54]:
base_model = ResNet50V2(
    weights='imagenet',
    input_shape=(224,224,3),
    include_top = False
)

In [38]:
base_model.trainable = True
cnt = 0
for layer in base_model.layers[:-80]:
    cnt+=1
    layer.trainable = False
# print(cnt)

In [39]:
print(train_generator.samples)  # Total images loaded
print(train_generator.class_indices)  # Class labels mapping


3780
{'antelope': 0, 'badger': 1, 'bat': 2, 'bear': 3, 'bee': 4, 'beetle': 5, 'bison': 6, 'boar': 7, 'butterfly': 8, 'cat': 9, 'caterpillar': 10, 'chimpanzee': 11, 'cockroach': 12, 'cow': 13, 'coyote': 14, 'crab': 15, 'crow': 16, 'deer': 17, 'dog': 18, 'dolphin': 19, 'donkey': 20, 'dragonfly': 21, 'duck': 22, 'eagle': 23, 'elephant': 24, 'flamingo': 25, 'fly': 26, 'fox': 27, 'goat': 28, 'goldfish': 29, 'goose': 30, 'gorilla': 31, 'grasshopper': 32, 'hamster': 33, 'hare': 34, 'hedgehog': 35, 'hippopotamus': 36, 'hornbill': 37, 'horse': 38, 'hummingbird': 39, 'hyena': 40, 'jellyfish': 41, 'kangaroo': 42, 'koala': 43, 'ladybugs': 44, 'leopard': 45, 'lion': 46, 'lizard': 47, 'lobster': 48, 'mosquito': 49, 'moth': 50, 'mouse': 51, 'octopus': 52, 'okapi': 53, 'orangutan': 54, 'otter': 55, 'owl': 56, 'ox': 57, 'oyster': 58, 'panda': 59, 'parrot': 60, 'pelecaniformes': 61, 'penguin': 62, 'pig': 63, 'pigeon': 64, 'porcupine': 65, 'possum': 66, 'raccoon': 67, 'rat': 68, 'reindeer': 69, 'rhinocer

In [40]:
for layer in base_model.layers:
  print(layer, layer.trainable)

<keras.src.engine.input_layer.InputLayer object at 0x7f25effd0070> False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x7f26493ae260> False
<keras.src.layers.normalization.batch_normalization.BatchNormalization object at 0x7f266e6ea1a0> False
<keras.src.layers.core.activation.Activation object at 0x7f26493ac7f0> False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x7f266e353fd0> False
<keras.src.layers.normalization.batch_normalization.BatchNormalization object at 0x7f266e351c30> False
<keras.src.layers.core.activation.Activation object at 0x7f266e352860> False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x7f266e351c00> False
<keras.src.layers.normalization.batch_normalization.BatchNormalization object at 0x7f266e54c580> False
<keras.src.layers.core.activation.Activation object at 0x7f266e54e0e0> False
<keras.src.layers.pooling.max_pooling2d.MaxPooling2D object at 0x7f266e54e5c0> False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x7f266e54d5

In [41]:
from tensorflow.keras.regularizers import l1

model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(64,activation='relu',kernel_regularizer=l1(0.0001)))
model.add(BatchNormalization())
model.add(Dropout(0.45))
model.add(Dense(90,activation='softmax'))

In [42]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 flatten_3 (Flatten)         (None, 51200)             0         
                                                                 
 batch_normalization_193 (B  (None, 51200)             204800    
 atchNormalization)                                              
                                                                 
 dropout_4 (Dropout)         (None, 51200)             0         
                                                                 
 dense_5 (Dense)             (None, 64)                3276864   
                                                                 
 batch_normalization_194 (B  (None, 64)                256       
 atchNormalization)                                   

In [43]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [45]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
path = "/kaggle/working/inception.h5"
callbacks = [
    ModelCheckpoint(filepath=path, save_best_only = True, verbose = 1, monitor='val_loss', mode='min'),
    EarlyStopping(monitor='val_loss', patience=5,verbose = 1,  mode='min', restore_best_weights=True),
    ReduceLROnPlateau(
    monitor='val_loss',  
    factor=0.5,         
    patience=3,          
    min_lr=1e-6,        
    verbose=1
)
]

In [47]:
history = model.fit(train_generator,epochs=10,validation_data=val_ds,callbacks=callbacks )

Epoch 1/10
119/119 [==============================] - ETA: 0s - loss: 5.4334 - accuracy: 0.2743
Epoch 1: val_loss improved from inf to 4.60748, saving model to /kaggle/working/inception.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


119/119 [==============================] - 74s 562ms/step - loss: 5.4334 - accuracy: 0.2743 - val_loss: 4.6075 - val_accuracy: 0.4667 - lr: 0.0010
Epoch 2/10
119/119 [==============================] - ETA: 0s - loss: 3.9185 - accuracy: 0.6032
Epoch 2: val_loss improved from 4.60748 to 3.34553, saving model to /kaggle/working/inception.h5
119/119 [==============================] - 64s 542ms/step - loss: 3.9185 - accuracy: 0.6032 - val_loss: 3.3455 - val_accuracy: 0.6840 - lr: 0.0010
Epoch 3/10
119/119 [==============================] - ETA: 0s - loss: 3.2840 - accuracy: 0.7066
Epoch 3: val_loss improved from 3.34553 to 2.84522, saving model to /kaggle/working/inception.h5
119/119 [==============================] - 64s 538ms/step - loss: 3.2840 - accuracy: 0.7066 - val_loss: 2.8452 - val_accuracy: 0.7747 - lr: 0.0010
Epoch 4/10
119/119 [==============================] - ETA: 0s - loss: 2.7162 - accuracy: 0.7960
Epoch 4: val_loss did not improve from 2.84522
119/119 [=====================

In [52]:
history2 = model.fit(train_generator,epochs=20,initial_epoch = 10, validation_data=val_ds,callbacks=callbacks )

Epoch 11/20
119/119 [==============================] - ETA: 0s - loss: 2.3216 - accuracy: 0.8942
Epoch 11: val_loss did not improve from 2.43233
119/119 [==============================] - 64s 534ms/step - loss: 2.3216 - accuracy: 0.8942 - val_loss: 2.8947 - val_accuracy: 0.7975 - lr: 0.0010
Epoch 12/20
119/119 [==============================] - ETA: 0s - loss: 2.2865 - accuracy: 0.9045
Epoch 12: val_loss did not improve from 2.43233
119/119 [==============================] - 64s 535ms/step - loss: 2.2865 - accuracy: 0.9045 - val_loss: 2.5503 - val_accuracy: 0.8296 - lr: 0.0010
Epoch 13/20
119/119 [==============================] - ETA: 0s - loss: 2.2178 - accuracy: 0.9108
Epoch 13: val_loss did not improve from 2.43233
119/119 [==============================] - 64s 534ms/step - loss: 2.2178 - accuracy: 0.9108 - val_loss: 2.9124 - val_accuracy: 0.8148 - lr: 0.0010
Epoch 14/20
119/119 [==============================] - ETA: 0s - loss: 2.4133 - accuracy: 0.9029
Epoch 14: val_loss did not 

KeyboardInterrupt: 

In [53]:
class_names = train_generator.class_indices

In [54]:
class_names

{'antelope': 0,
 'badger': 1,
 'bat': 2,
 'bear': 3,
 'bee': 4,
 'beetle': 5,
 'bison': 6,
 'boar': 7,
 'butterfly': 8,
 'cat': 9,
 'caterpillar': 10,
 'chimpanzee': 11,
 'cockroach': 12,
 'cow': 13,
 'coyote': 14,
 'crab': 15,
 'crow': 16,
 'deer': 17,
 'dog': 18,
 'dolphin': 19,
 'donkey': 20,
 'dragonfly': 21,
 'duck': 22,
 'eagle': 23,
 'elephant': 24,
 'flamingo': 25,
 'fly': 26,
 'fox': 27,
 'goat': 28,
 'goldfish': 29,
 'goose': 30,
 'gorilla': 31,
 'grasshopper': 32,
 'hamster': 33,
 'hare': 34,
 'hedgehog': 35,
 'hippopotamus': 36,
 'hornbill': 37,
 'horse': 38,
 'hummingbird': 39,
 'hyena': 40,
 'jellyfish': 41,
 'kangaroo': 42,
 'koala': 43,
 'ladybugs': 44,
 'leopard': 45,
 'lion': 46,
 'lizard': 47,
 'lobster': 48,
 'mosquito': 49,
 'moth': 50,
 'mouse': 51,
 'octopus': 52,
 'okapi': 53,
 'orangutan': 54,
 'otter': 55,
 'owl': 56,
 'ox': 57,
 'oyster': 58,
 'panda': 59,
 'parrot': 60,
 'pelecaniformes': 61,
 'penguin': 62,
 'pig': 63,
 'pigeon': 64,
 'porcupine': 65,
 'pos

In [60]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image

def preprocess_image(img_path, target_size=(224, 224)):  
    img = image.load_img(img_path, target_size=target_size)  # Resize image
    img_array = image.img_to_array(img)  # Convert to array (shape: (224, 224, 3))
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension (1, 224, 224, 3)
    img_array = img_array / 255.0  # Normalize if model requires it
    return img_array


In [ ]:
import os

In [ ]:
img_path = "/kaggle/input/animal-image-dataset-90-different-animals/animals/animals/bee/0d562e4829.jpg"
img_array = preprocess_image(img_path)
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions)
# predicted_label = class_names[predicted_class]
# print(f"{img_name} → Predicted Class: {predicted_class}")
predicted_class

In [28]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Define the training data directory
train_dir = "/kaggle/input/animal-image-dataset-90-different-animals/animals/animals"

# Load training dataset without shuffling
train_ds = image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),  # Resize to match the model input size
    batch_size=32,
    shuffle=False  # IMPORTANT: Don't shuffle to maintain order
)

Found 5400 files belonging to 90 classes.


In [31]:
# Extract true labels
class_names = train_ds.class_names  # Get class labels from dataset
y_train_true = []  # Store actual labels

for images, labels in train_ds:
    y_train_true.extend(labels.numpy())  # Convert tensor to numpy array


In [29]:
import numpy as np

# Get model predictions
y_train_pred_probs = model.predict(train_ds)  # Get probabilities
y_train_pred = np.argmax(y_train_pred_probs, axis=1)  # Convert to class labels


169/169 [==============================] - 18s 90ms/step


In [1]:
!pip install tensorflow==2.15.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.7 MB/s eta 0:00:00:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 5.5 MB/s eta 0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 99.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.0
    Uninstalling wrapt-1.17.0:
      Successfully uninstalled wrapt-1.17.0
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstalled keras-3.5.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
  Attempting uninstall: ml-d

In [ ]:
optimizer=RMSprop(learning_rate=0.001)